Nota: Escrito sin acentos ni egnes.

Simulacion del modelo autoregresivo con marginales Gamma(a,1), y estimacion por medio del algoritmo Expectation-Maximization (EM) con calulo de esperanzas condicionales utilizando al algoritmo de Markov Chain Monte Carlo (MCMC) Metropolis Hastings.

Revisar archivo pdf para detalles teoricos y resultados de las simulaciones.

In [4]:
using Random
using SpecialFunctions
using Distributions
using StatsBase

Primero se define una function llamada target, donde se van a dar como argumentos a, phi, x, y1 y y2 y se devuelve el logaritmo de la distribucion objetivo de la cual se quieren estimar esperanzas en el algoritmo MCMC Metropolis Hastings. y1 y y2 representan la observacion at tiempo t y al tiempo t+1 respectivamente.

In [6]:
function target(x,y1,y2,a,phi)
  return (a+x)*log(1+phi) - lgamma(a+x) + (a+x-1)*log(y2) - (1+phi)*y2 - (phi*y1) + x*log(phi*y1) - lgamma(x+1)
end

target (generic function with 1 method)

Ahora se define la funcion MCMC con la cual se construye la cadena de Markov con la distribucion target deseada.

In [7]:
function MCMC(N,a,phi,y1,y2)
    #iter=1
    x=zeros(N)
#Valor inicial de x a partir de una Poisson
    lambda=phi*mean([y1,y2,a])
    x[1]=rand( Poisson(lambda))
    #Se utiliza un for para iterar hasta llegar al numero N de iteraciones deseadas
    for i in 2:N
        x_prop=rand( Poisson(lambda))
        #Calcular la probabilidad de aceptacion
        #Se aplica logaritmo para evitar problemas numericos
        log_alfa = target(x_prop,y1,y2,a,phi) - target(x[i-1],y1,y2,a,phi) + log(pdf(Poisson(lambda),x[i-1])) - log(pdf(Poisson(lambda),x_prop))
        #Se actualiza x y se almacena en un vector para formar la cadena
        u=rand()
        x[i]=x_prop*(log(u)<=log_alfa)*1 + x[i-1]*(log(u)>log_alfa)*1
    end
    return x
end

MCMC (generic function with 1 method)

Ahora se define la funcion que genera trayectorias del proceso (Yt,Xt), con el fin de poner a prueba el algoritmo EM y estimar parametros a partir de series simuladas.

In [8]:
function serie(a,phi,M)
    #La primera observacion se muestrea de la distribucion estacionaria Gamma(a,1)
    y=rand(Gamma(a,1))
    #Se genera x a partir de la distribucion condicional
    x=rand(Poisson(phi*y))
    i=1
    #se repite el procedimiento hasta alcanzar el tamano de trayectoria deseado
  while i < M 
    y=[y;rand(Gamma(a+x[length(x)],1/(1+phi)))]
    x=[x;rand(Poisson(phi*y[length(y)]))]
    i=i+1
    end
    return hcat(y,x)
end

serie (generic function with 1 method)

El paso siguiente es hacer la estimacion de las esperanzas condicionales involucradas, a traves de una funcion llama condicionales, que toma como parametros N el tamano total de las cadenas a ser usadas en la estimacion, M el periodo burn-in, $a$ y $\phi$ los parametros, y $y$ la serie de tiempo.

In [11]:
function condicionales(N,M,a,phi,y)
    x=zeros(length(y)-1)
    for i in 1:length(x)
    #Se aplica la funcion MCMC a cada par de observaciones consecutivas de la serie
    cadena=MCMC(N,a,phi,y[i],y[i+1])[M+1:N]
    x[i]=mean(cadena)
    end
  return x
end

condicionales (generic function with 1 method)

Posteriormente se ejecuta el algoritmo EM utilizando una funcion llamada EM. Los parametros que toma la funcion son $a$, el valor inicial de $\phi$, la serie de tiempo $y$, N y M el tamano y burn-in de las cadenas de Markov empleadas en el MCMC, I el numero maximo de iteraciones en el algoritmo EM, y Q el numero de terminos incluidos en el promedio movil para definir cuando detener el algoritmo. La funcion devuelve una lista, cuyo primer elemento es un vector con los valores de $\phi$ en cada iteracion, y el segundo elemento de la lista es un vector con los valores de los promedios.

In [16]:
function EM(a,phi_ini,y,N,M,I,Q)
#Paso 1-Actualizar el parametro por primera vez
    #se estiman las esperanzas condicionales
    cond=condicionales(N,M,a,phi_ini,y)
    K1=(length(y)-1)*a + 2*sum(cond)
    K2=sum(cond)
    K3=2*sum(y) - y[1] - y[length(y)]
    #se actualiza el valor de phi maximizando la esperanza condicional de la log-verosimilitud completa
    fi=((K1-K3)+sqrt((K1-K3)^2 + 4*K3*K2))/(2*K3)  
    phi=[phi_ini;fi]
    ma=zeros(I)
    #Esta funcion imprime cada que acaba una iteracion para poder observar como va creciendo o decreciendo 
    #el valor de phi, y del promedio movil, y saber en que iteracion va el algoritmo
    #y que tan rapida es cada iteracion
    print("It ")
    print(1)
    print(" = ")
    print(fi)
    print(", MA ")
    print(" = ")
    print(ma[1])
 #Paso i   
  for i in 2:Q
    cond=condicionales(N,M,a,phi[i],y)
    K1=(length(y)-1)*a + 2*sum(cond)
    K2=sum(cond)
    fi=((K1-K3)+sqrt((K1-K3)^2 + 4*K3*K2))/(2*K3)
    phi=[phi;fi]
    print(", It ")
    print(i)
    print(" = ")
    print(fi)
    print(", MA ")
    print(" = ")
    print(ma[i])    
    end
    for i in Q+1:Q+2
    cond=condicionales(N,M,a,phi[i],y)
    K1=(length(y)-1)*a + 2*sum(cond)
    K2=sum(cond)
    fi=((K1-K3)+sqrt((K1-K3)^2 + 4*K3*K2))/(2*K3)
    phi=[phi;fi]
    ma[i]=mean(phi[i-Q:end])
    print(", It ")
    print(i)
    print(" = ")
    print(fi)
    print(", MA ")
    print(" = ")
    print(ma[i])
    end
    for i in Q+3:I
    cond=condicionales(N,M,a,phi[i],y)
    K1=(length(y)-1)*a + 2*sum(cond)
    K2=sum(cond)
    fi=((K1-K3)+sqrt((K1-K3)^2 + 4*K3*K2))/(2*K3)
    phi=[phi;fi]
    ma[i]=mean(phi[i-Q:end])
    print(", It ")
    print(i)
    print(" = ")
    print(fi)
    print(", MA ")
    print(" = ")
    print(ma[i])
        if sign(ma[i]-ma[i-1])!=sign(ma[i-1]-ma[i-2])
            
            break
        end
    end
  return [phi,ma[ma.!=0]]
end

EM (generic function with 1 method)